In [ ]:
! pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
! pip install pdf2image

In [ ]:
! sudo apt install tesseract-ocr
! pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (10.9 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [ ]:
!apt-get install poppler-utils  #для google colab

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 0s (1,253 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121965 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
! pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.8 MB/s eta 0:00:00


In [ ]:
import PyPDF2
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
import os

In [ ]:
from tqdm import tqdm

Следующие функции выполняют прочтения pdf файла и картинок, если не распознаётся текст

In [ ]:
def text_extraction(element):
    line_text = element.get_text()

    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            for character in text_line:
                if isinstance(character, LTChar):
                    line_formats.append(character.fontname)
                    line_formats.append(character.size)
    format_per_line = list(set(line_formats))

    return (line_text, format_per_line)

In [ ]:
def crop_image(element, pageObj):
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1]
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = 'PDF_image.png'
    image.save(output_file, 'PNG')

def image_to_text(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

In [ ]:
def read_text_pdf(pdf_path):

    pdfFileObj = open(pdf_path, 'rb')
    pdfReaded = PyPDF2.PdfReader(pdfFileObj)
    text_per_page = {}
    image_flag = False

    for pagenum, page in tqdm(enumerate(extract_pages(pdf_path))):

        pageObj = pdfReaded.pages[pagenum]
        page_text = []
        line_format = []
        page_content = []
        text_from_images = []

        page_elements = [(element.y1, element) for element in page._objs]
        page_elements.sort(key=lambda a: a[0], reverse=True)

        for i,component in enumerate(page_elements):
                element = component[1]

                if isinstance(element, LTTextContainer):
                    (line_text, format_per_line) = text_extraction(element)
                    page_text.append(line_text)
                    line_format.append(format_per_line)
                    page_content.append(line_text)

                if isinstance(element, LTFigure):
                    crop_image(element, pageObj)
                    convert_to_images('cropped_image.pdf')
                    image_text = image_to_text('PDF_image.png')
                    text_from_images.append(image_text)
                    page_content.append(image_text)
                    page_text.append('image')
                    line_format.append('image')
                    image_flag = True

        dctkey = 'Page_'+str(pagenum)
        text_per_page[dctkey]= [page_text, line_format, page_content]

    pdfFileObj.close()
    if image_flag:
        os.remove('cropped_image.pdf')
        os.remove('PDF_image.png')

    return text_per_page

Получается словарь, где ключи — страницы, а значения — текст на странице.

In [ ]:
pdf_path = 'Alpharts Tod. Dietrich und Wenezlan только текст.pdf'

In [ ]:
text = read_text_pdf(pdf_path)

84it [00:07, 11.40it/s]


In [ ]:
import re

В последующей функции убираются примечания к тексту в самом конце строки, номера страниц вверху. Также с помощью регулярных выражений заменяется строчки спропусками на соседние из левой колонки. Последнее характерно для произведения Dietrich und Wenezlan

In [ ]:
name = 'Alpharts Tod.txt'

In [ ]:
with open(name, 'a') as f:
  for blok in text:
    if blok == 'Page_65':
      break
    else:
        for stroka in text[blok][0]:
          stroka = stroka.replace('Æ', '(')
          stroka = stroka.replace('æ', ')')
          if not any(ch.isdigit() for ch in stroka) and stroka != '‹Alpharts Tod›\n':
            f.write(stroka)
    f.write('\n')

In [ ]:
with open(name, 'a') as f:
  for blok in text:
    if blok[-2] != '_':
      if int(blok[-2:]) > 64:
        stroka_or = []
        stroka_predp = []
        final_stroki = []
        for stroka in text[blok][0]:
          stroka = stroka.replace('Æ', '(')
          stroka = stroka.replace('æ\n', ')\n')
          stroka = stroka.replace('æ»', ')»')
          stroka = stroka.replace('æ,', '),')
          stroka = stroka.replace('æ.', ').')
          stroka = stroka.replace('æ!', ')!')
          stroka = stroka.replace('æ?', ')?')
          if not any(ch.isdigit() for ch in stroka) and stroka != '‹Dietrich und Wenezlan›\n':
            if 'Zp)' in stroka or '(G)' in stroka:
              stroka_predp.append(stroka)
            else:
              stroka_or.append(stroka)

        for element in stroka_or[0].split('\n'):
          flag = 0
          if element != '':
            for element_2 in stroka_predp:

              if element[1:8] in element_2:
                flag = 1
                s = re.sub('\([^)]*\)', '', element_2)
                f.write(s)

          if '(…)' not in element and flag != 1:
              s2 = element.replace('(', '')
              s2 = s2.replace(')', '')
              f.write(s2)
          elif '(…)' in element and flag != 1:
            f.write(element)
        f.write('\n')